In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
dataset = pd.read_csv('/home/hanwenli/work/2025/AL_SSL/results_with_num_unlabeled_llama3.csv')
trivia_qa_df = dataset[dataset['dataset'] == 'trivia_qa']
trivia_qa_df

In [ ]:
data_list = [
    {'name':'SAR','value':trivia_qa_df[trivia_qa_df['uncertainty_type']=='sar']['auroc'].to_list()[:-7], 'num_unlabeled':trivia_qa_df[trivia_qa_df['uncertainty_type']=='sar']['unlabeled_size'].to_list()[:-7]},
    {'name':'SE','value':trivia_qa_df[trivia_qa_df['uncertainty_type']=='semanticentropy']['auroc'].to_list()[:-7], 'num_unlabeled':trivia_qa_df[trivia_qa_df['uncertainty_type']=='semanticentropy']['unlabeled_size'].to_list()[:-7]},
    {'name':'MSP','value':trivia_qa_df[trivia_qa_df['uncertainty_type']=='maximumsequenceprobability']['auroc'].to_list()[:-7], 'num_unlabeled':trivia_qa_df[trivia_qa_df['uncertainty_type']=='maximumsequenceprobability']['unlabeled_size'].to_list()[:-7]},
    {'name':'LS','value':trivia_qa_df[trivia_qa_df['uncertainty_type']=='lexicalsimilarity']['auroc'].to_list()[:-7], 'num_unlabeled':trivia_qa_df[trivia_qa_df['uncertainty_type']=='lexicalsimilarity']['unlabeled_size'].to_list()[:-7]},
    {'name':'MCSE','value':trivia_qa_df[trivia_qa_df['uncertainty_type']=='montecarlosequenceentropy']['auroc'].to_list()[:-7], 'num_unlabeled':trivia_qa_df[trivia_qa_df['uncertainty_type']=='montecarlosequenceentropy']['unlabeled_size'].to_list()[:-7]}
]
data_list

In [ ]:
def plot_uncertainty_types(data_list, output_filename='uncertainty_auroc_plot.png'):
    """
    Plot AUROC values for different uncertainty types with unlabeled size as x-axis
    
    Parameters:
    data_list: list of dictionaries with 'name', 'value', and 'num_unlabeled' keys
    output_filename: str, filename to save the plot
    """
    import matplotlib.pyplot as plt
    import numpy as np
    
    colors = sns.color_palette('colorblind', len(data_list))

    
    # Define colors and markers for each uncertainty type
    style_settings = {
        'SAR': {'color': '#1b263b', 'marker': 'o'},
        'SE': {'color': '#3a6ea5', 'marker': 'o'},
        'MSP': {'color': '#778da9', 'marker': 'o'},
        'LS': {'color': '#C4C4C4', 'marker': 'o'},
        'MCSE': {'color': '#bf0000', 'marker': 'o'}
    }
    
    # Create a figure with sufficient size
    plt.figure(figsize=(10, 6))
    
    # Plot each uncertainty type
    for i, item in enumerate(data_list):
        name = item['name']
        values = item['value']
        num_unlabeled = item['num_unlabeled']
        
        # Get style settings
        style = style_settings.get(name, {'color': 'black', 'marker': 'x'})
        
        # Plot the line
        plt.plot(
            num_unlabeled,
            values,
            label=name,
            color=colors[i],
            marker=style['marker'],
            linestyle='-',
            linewidth=2
        )
    
    # Set the x-axis and y-axis labels
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    # Add grid for better readability
    plt.grid(True)
    
    # Add a legend
    plt.legend(fontsize=20, loc='lower right')
    
    # Adjust layout to make sure everything fits
    
    # Save the figure
    plt.savefig(output_filename, dpi=300, bbox_inches='tight')
    
    # Display the chart
    plt.show()

# Example usage with your data
plot_uncertainty_types(data_list, 'uncertainty_auroc_plot.png')

In [ ]:
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.int8)
model.config.pad_token_id = 128001
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
prompt = "Answer the following question concisely: What is the capital of France?\n Answer: "

inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)
attention_mask = inputs.attention_mask



generate_ids = model.generate(inputs.input_ids, max_new_tokens=inputs.input_ids.shape[1]+5, attention_mask=attention_mask)
answer = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)

print(answer)

In [ ]:
generate_ids = model.generate(inputs.input_ids, max_new_tokens=inputs.input_ids.shape[1]+5, attention_mask=attention_mask)
answer = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)

print(answer)

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(
    model_id, device_map="auto", torch_dtype=torch.float16)

tokenizer = AutoTokenizer.from_pretrained(model_id)
input_text = "Answer the following question concisely: What is the capital of France?\n Answer: "
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

output = model.generate(**input_ids, max_new_tokens=10)

print(tokenizer.decode(output[0], skip_special_tokens=True))


/home/hanwenli/anaconda3/envs/debug/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 30/30 [00:40<00:00,  1.36s/it]
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Answer the following question concisely: What is the capital of France?
 Answer:  Paris


In [3]:
simple_qa = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n" + "Answer the question concisely." + "Q: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
simple_qa = simple_qa.format(question="What is the capital of France?")
print(simple_qa)
input_ids = tokenizer(simple_qa, return_tensors="pt").to("cuda")


<|begin_of_text|><|start_header_id|>user<|end_header_id|>
Answer the question concisely.Q: What is the capital of France?<|eot_id|><|start_header_id|>assistant<|end_header_id|>


In [4]:
generate_ids = model.generate(input_ids.input_ids, max_new_tokens=10, attention_mask=input_ids.attention_mask)
answer = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)

print(answer)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


['user\nAnswer the question concisely.Q: What is the capital of France?assistant\n\nParis']
